In [1]:
import csv
import pandas as pd
import random
import copy
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
import sys
from matching.games import HospitalResident
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)


# fiktivni uchazeci - viz skritpt main/vyroba_testovacich_dat.ipynb
deti = pd.read_csv('../deti.csv', dtype = {
    'dite_id': 'string', 
    'spadova_skolka': 'string', 
    'skolka_sourozence': 'int64'})

# skutecne skolky
skolky = pd.read_csv('../skolky.csv', dtype = {
    'skolka_id': 'string'})

# parametry tohoto testu:
random.seed(11)
pocet_deti = 10
pocet_skolek = 4
kapacita_jedne_skolky = 2   # volitelny udaj. Pri zakomentovani se pouziji skutecne kapacity.
skolky['volna_mista'] = kapacita_jedne_skolky  # volitelny udaj. Pri zakomentovani se pouziji skutecne kapacity.


def get_random_rows_skolky(dataframe, num_rows):
    # vybere nahodne skolky z datasetu 'skolky' v poctu pocet_skolek
    random_rows = dataframe.sample(n=num_rows)
    random_rows.reset_index()
    return random_rows
skolky = get_random_rows_skolky(skolky, num_rows = pocet_skolek)

skolky.to_csv('skolky_test.csv')

def get_random_rows_deti(dataframe, num_rows):
    # Vybere nahodne uchazece z datasetu 'deti' v poctu pocet_uchazecu
    random_rows = dataframe.sample(n=num_rows)
    random_rows['spadova_skolka'] = [random.choice(list(skolky.skolka_id)) for _ in range(num_rows)]
    random_rows['skolka_sourozence'] = [int(random.choice(list(skolky.skolka_id))) if original_value >= 0 else original_value
        for original_value in random_rows['skolka_sourozence']]
    random_rows.reset_index()
    return random_rows
    
deti = get_random_rows_deti(deti, num_rows = pocet_deti)


deti.to_csv('deti_test.csv')

# vyroba prihlasek:
def get_prihlasky(deti_df, skolky_df):
    # Fuknce simuluje výběr oblíbených školek. Počet položek je náhodný.
    # Funkce při "výběru" oblíbených školek zohledňuje spádovost a školku sourozence.
    data = {'dite': [], 'jmeno': [], 'skolka': [], 'poradi': []}
    for idx, row in deti_df.iterrows():
        vybrane_skolky = set()
        pocet_prihlasek = random.randint(1, len(skolky_df))
        dostupne_skolky= list(skolky_df.skolka_id)
        dostupne_skolky = [int(_) for _ in dostupne_skolky]
        for i in range(pocet_prihlasek):
            if i == 0:
                choice = int(row['spadova_skolka'])
                vybrane_skolky.add(choice)
                dostupne_skolky = [_ for _ in dostupne_skolky if _ != choice]
            elif (i == 1 and row['skolka_sourozence'] >= 0):
                choice = int(row['skolka_sourozence'])
                vybrane_skolky.add(choice) 
                dostupne_skolky = [_ for _ in dostupne_skolky if _ != choice]
            else:
                random.shuffle(dostupne_skolky)
                choice = dostupne_skolky[0]
                vybrane_skolky.add(choice)
                dostupne_skolky = [_ for _ in dostupne_skolky if _ != choice]
    
        vybrane_skolky = list(vybrane_skolky)
        vybrane_skolky = [str(_) for _ in vybrane_skolky]
    
        for index, value in enumerate(vybrane_skolky):
            data['dite'].append(str(row['dite_id']))
            data['jmeno'].append(row['jmeno'])
            data['skolka'].append(value)
            data['poradi'].append(index + 1)

    df = pd.DataFrame(data)
    return df

prihlasky = get_prihlasky(deti, skolky)
prihlasky.to_csv('prihlasky.csv')

print('Vstupní data pro rozřazení (vybrané sloupce): ')
print()
print(deti[['dite_id', 'jmeno', 'datum_narozeni', 'spadova_skolka']].head(10))
print()
print(skolky[['skolka_id', 'nazev_kratky', 'volna_mista']].head(10))
print()
print(prihlasky)

Vstupní data pro rozřazení (vybrané sloupce): 

     dite_id                   jmeno datum_narozeni spadova_skolka
4248    4248  Dominika Test Mrázková     2020-12-04             61
3963    3963    Zuzana Test Marešová     2020-08-21             61
2233    2233     Miloslav Test Polák     2019-03-22             61
3636    3636   Petra Test Havlíčková     2019-11-13             44
857      857       Josef Test Veselý     2021-04-20             44
2357    2357       Josef Test Veselý     2019-01-01             61
4963    4963  Štěpánka Test Marešová     2017-11-19             44
4855    4855  Marie Test Procházková     2018-11-06            109
3831    3831    Tereza Test Benešová     2017-11-03             61
3521    3521  Patricia Test Urbanová     2019-10-21             94

    skolka_id     nazev_kratky  volna_mista
106       109      MŠ Šrámkova            2
44         44  MŠ Jana Broskvy            2
91         94        MŠ Pšeník            2
60         61       MŠ Křenová        

In [2]:
# vypocet bodu pro jednotlive uchazece podle kritérií popsaných zde: https://zapisdoms.brno.cz/kriteria-rizeni

def get_mestska_cast(deti_df): 
    # Funkce rozšíří tabulku dětí o sloupec "mestska_cast", převzatý od spádové školky, za účelem obodování. 
    deti_df = pd.merge(deti_df, skolky[['skolka_id', 'mc']], left_on = 'spadova_skolka', right_on = 'skolka_id')  
    deti_df.drop('skolka_id', axis = 1, inplace = True)
    return deti_df
deti = get_mestska_cast(deti)

def get_age(birthday):
    # Z data narození vypočte věk v letech a dnech k letošnímu 31. srpnu.
    today = datetime.date.today()
    current_year = today.year
    schoolyear_start = datetime.date(birthday.year, 8, 31)
    difference = schoolyear_start - birthday - relativedelta(days = 1)   # prizpusobeni webu zapisdoms.brno.cz

    schoolyear_start_current = datetime.date(current_year, 8, 31)
    age_in_years = int(round((schoolyear_start_current - schoolyear_start).days/365, 0))
    if difference.days < 0:
        age_in_years -= 1
    return age_in_years, difference.days

def get_points_years(age, age_difference_days):
    # Přidelí body za věk.
    options = {7: 2160, 6: 2120, 5: 2080, 4: 2040, 3: 2000, 2: 0, 1: 0}
    calculate_points = lambda x: 1000 if (x == 2 and age_difference_days < 0) else options[x]
    points_years = calculate_points(age)
    return points_years

def calculate_points_one_child(id_child):
    # Sečte body za všechna bodovaná kritéria.

    # vytvořit tabulku "1 uchazeč, všechny školky"  
    copy_skolky = copy.deepcopy(skolky)
    copy_skolky['dite_id'] = id_child
    df_one_child = pd.merge(copy_skolky, deti, left_on ='dite_id', right_on = 'dite_id', how = 'left')
    df_one_child.rename(columns={'mc_x': 'mc_skolka', 'mc_y': 'mc_dite'}, inplace = True)

    # body za věk
    df_one_child['datum_narozeni'] = pd.to_datetime(df_one_child['datum_narozeni']).dt.date
    df_one_child['vek'] = df_one_child['datum_narozeni'].apply(lambda x: pd.Series(get_age(x)))[0]
    df_one_child['vek_dny_srpen31'] = df_one_child['datum_narozeni'].apply(lambda x: pd.Series(get_age(x)))[1]
    df_one_child['body_za_vek_roky'] = df_one_child.apply(lambda row: get_points_years(row['vek'], row['vek_dny_srpen31']), axis=1)
    df_one_child['body_za_vek_dny'] = df_one_child.apply(lambda row: 0 if (row['vek_dny_srpen31'] < 0) else row['vek_dny_srpen31']*0.02, axis = 1)
    df_one_child['prioritni_vek'] = df_one_child.apply(lambda row: True if (3 <= row['vek'] <= 6) else False, axis = 1)

    # body za bydliště
    df_one_child['body_spadovost'] = df_one_child.apply(lambda row: 250 if row['bydliste_brno'] == True else 0, axis = 1)
    df_one_child['body_spadovost'] = df_one_child.apply(lambda row: 500 if row['mc_skolka'] == row['mc_dite'] else row['body_spadovost'], axis = 1)
    df_one_child['body_spadovost'] = df_one_child.apply(lambda row: 750 if (row['skolka_id'] == row['spadova_skolka']) and (row['prioritni_vek'] == False) else row['body_spadovost'], axis = 1)
    df_one_child['body_spadovost'] = df_one_child.apply(lambda row: 1000 if (row['skolka_id'] == row['spadova_skolka']) and (row['prioritni_vek'] == True) else row['body_spadovost'], axis = 1)

    # body za sourozence ve školce
    df_one_child['skolka_sourozence'] = pd.to_numeric(df_one_child['skolka_sourozence'], errors='coerce') # nutne kvuli df.query nize
    df_one_child['skolka_id'] = pd.to_numeric(df_one_child['skolka_id'], errors='coerce')
    df_one_child['body_sourozenec'] = df_one_child.query('prioritni_vek == True and skolka_sourozence == skolka_id').apply(lambda x: 10, axis = 1)
    df_one_child['body_sourozenec'].fillna(0, inplace = True)
    df_one_child['skolka_sourozence'] = df_one_child['skolka_sourozence'].astype(str)
    df_one_child['skolka_id'] = df_one_child['skolka_id'].astype(str)

    # dve specialni skolky, které mají jako spádovou oblast celé Brno a nabízejí prodloužený provoz
    # body navíc, pokud uchazeč doloží potřebu prodlouženého provozu
    # zapisdoms.brno.cz nespecifikuje počet bodů navíc >>> arbitrárně stanoveno 50 bodů
    df_one_child.loc[df_one_child['mc_skolka'] == "brno",'body_spadovost'] = df_one_child.loc[df_one_child['mc_skolka'] == "brno", 'body_spadovost'].apply(lambda x: 1000)
    df_one_child.loc[df_one_child['mc_skolka'] == "brno",'body_sourozenec'] = df_one_child.loc[df_one_child['mc_skolka'] == "brno", 'body_sourozenec'].apply(lambda x: 10)
    df_one_child['body_prodlouz_provoz'] = 0
    df_one_child.loc[df_one_child['mc_skolka'] == "brno",'body_prodlouz_provoz'] = df_one_child.loc[df_one_child['mc_skolka'] == "brno", 'body_prodlouz_provoz'].apply(lambda x: 50 if 'prodlouzena_dochazka' == True else 0)

    # nezohledneni bodů za "Den věku dítěte v roce narození" v případě spádových školek
    df_one_child.loc[df_one_child['spadova_skolka'] == df_one_child['skolka_id'], 'body_za_vek_dny'] = df_one_child.loc[df_one_child['spadova_skolka'] == df_one_child['skolka_id'], 'body_za_vek_dny'].apply(lambda x: 0)

    # součet bodů
    df_one_child['body_soucet'] = df_one_child['body_sourozenec'] + df_one_child['body_spadovost'] + df_one_child['body_za_vek_roky'] + df_one_child['body_za_vek_dny'] + df_one_child['body_prodlouz_provoz']
    d_pivoted = df_one_child.pivot(index = 'dite_id', columns = 'skolka_id', values = 'body_soucet')
    d_pivoted = pd.merge(deti, d_pivoted, how = 'right', left_on = 'dite_id', right_on = 'dite_id')
    d_pivoted.to_csv('d_pivoted.csv')
    return d_pivoted
    
# výstup pro prvního uchazeče:
body = calculate_points_one_child(deti.iloc[0]['dite_id'])  

# výstup pro všechny uchazeče:
for i in range(1, len(deti)):
    one_child_pivoted_df = calculate_points_one_child(deti.iloc[i]['dite_id'])
    body = pd.concat([body, one_child_pivoted_df])   
body[['dite_id', 'spadova_skolka', 'skolka_sourozence']] = body[['dite_id', 'spadova_skolka', 'skolka_sourozence']].astype('str')
body.to_csv('body.csv')


print('Všichni uchazeči vs. všechny školky: přidělené body')
print()
seznam_id_skolek = list(skolky.skolka_id)
print(body[['dite_id', 'jmeno'] + seznam_id_skolek])



Všichni uchazeči vs. všechny školky: přidělené body

  dite_id                   jmeno      109       44       94       61
0    4248  Dominika Test Mrázková  2250.00  2250.00  2500.00  3000.00
0    3963    Zuzana Test Marešová  2290.18  2290.18  2540.18  3040.00
0    2233     Miloslav Test Polák  2333.22  2333.22  2583.22  3080.00
0    2357       Josef Test Veselý  2334.82  2334.82  2584.82  3080.00
0    3831    Tereza Test Benešová  2370.00  2380.00  2620.00  3120.00
0    3636   Petra Test Havlíčková  2040.00  3040.00  2040.00  2040.00
0     857       Josef Test Veselý  2252.64  3000.00  2252.64  2252.64
0    4963  Štěpánka Test Marešová  2370.00  3120.00  2370.00  2370.00
0    4855  Marie Test Procházková  3080.00  2330.00  2330.00  2330.00
0    3521  Patricia Test Urbanová  2290.00  2290.00  3040.00  2540.00
